In [85]:
import pandas as pd
import numpy as np

%pip install xlrd

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\Albin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [86]:
gez = pd.read_csv("gezondheid-2020.csv", sep=";", index_col="Codering_3")
kwb = pd.read_excel("kwb-2019.xls", index_col="gwb_code_10")


In [87]:
# Find the amount rows that does not exists in gez and kwb
miss_aligned_codes = set(gez.index) - set(kwb.index)
print(
    f"{len(miss_aligned_codes)} codes are missing from on or the other dataset. That equates to approximately {len(miss_aligned_codes) / len(gez.index) * 100:.3f}% of the codes."
)

# Drop the rows where the code is missing
kwb = kwb.drop(miss_aligned_codes, errors="ignore")
gez = gez.drop(miss_aligned_codes, errors="igore")


524 codes are missing from on or the other dataset. That equates to approximately 3.022% of the codes.


## Load kwb-dataset


In [88]:
kwb.sample(5)


,gwb_code_8,regio,gm_naam,recs,gwb_code,ind_wbi,a_inw,a_man,a_vrouw,a_00_14,...,g_afs_kv,g_afs_sc,g_3km_sc,a_opp_ha,a_lan_ha,a_wat_ha,pst_mvp,pst_dekp,ste_mvs,ste_oad
gwb_code_10,,,,,,,,,,,,,,,,,,,,,
BU03638702,3638702,Delflandpleinbuurt Oost,Amsterdam,Buurt,BU03638702,1,1555,850,700,225,...,"0,4","0,5","31,8",18,17,1,1059,1,1,5715
BU05050507,5050507,Standhasenstraat en omgeving,Dordrecht,Buurt,BU05050507,1,1370,680,690,250,...,"0,2","0,2","24,6",15,15,0,3312,1,2,2418
BU19110104,19110104,Wateringskant,Hollands Kroon,Buurt,BU19110104,1,65,35,35,10,...,"2,5","2,5","0,8",153,153,0,1735,4,5,47
GM0086,86,Opsterland,Opsterland,Gemeente,GM0086,.,29723,14950,14773,4787,...,"1,0","0,9","2,5",22764,22440,324,.,.,5,403
BU01710323,1710323,Luttelgeest-landelijk gebied,Noordoostpolder,Buurt,BU01710323,1,1185,695,490,215,...,"2,5","1,5","1,0",3805,3762,44,8315,1,5,66


## Load gezondheid-dataset


In [89]:
gez.sample(5)


,ID,Leeftijd,Marges,WijkenEnBuurten,Perioden,Gemeentenaam_1,WekelijkseSporters_6,Eenzaam_28
Codering_3,,,,,,,,
WK164008,27647,20300,MW00000,WK164008,2020JJ00,Leudal,46.7,43.5
BU01060812,5471,20300,MW00000,BU01060812,2020JJ00,Assen,60.4,38.0
BU02740303,36419,20300,MW00000,BU02740303,2020JJ00,Renkum,43.0,51.1
BU06320204,49736,20300,MW00000,BU06320204,2020JJ00,Woerden,56.6,39.7
BU08280503,34877,20300,MW00000,BU08280503,2020JJ00,Oss,.,.


In [90]:
# Drop unnecessary columns
gez = gez.drop(
    ["ID", "Leeftijd", "Marges", "Perioden", "Gemeentenaam_1", "WijkenEnBuurten"],
    axis=1,
    errors="ignore",
)


In [91]:
gez.info()


<class 'pandas.core.frame.DataFrame'>
Index: 16817 entries, GM1680 to BU01935210
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   WekelijkseSporters_6  16817 non-null  object
 1   Eenzaam_28            16817 non-null  object
dtypes: object(2)
memory usage: 394.1+ KB


In [92]:
# Fix wrong dtype on the columns
wrong_dtype = ["WekelijkseSporters_6", "Eenzaam_28"]
gez[wrong_dtype] = gez[wrong_dtype].replace(" .", np.nan)
gez[wrong_dtype] = gez[wrong_dtype].astype(float)


In [93]:
round(gez.describe(), 2)


,WekelijkseSporters_6,Eenzaam_28
count,16127.00,16127.00
mean,48.52,44.08
std,7.70,6.38
min,14.70,22.30
25%,43.30,39.40
50%,48.10,42.70
75%,53.50,47.70
max,75.40,79.50


In [94]:
gez.isna().sum() / len(gez) * 100


WekelijkseSporters_6    4.102991
Eenzaam_28              4.102991
dtype: float64

# Combine the Data

In [95]:
combined = kwb.copy()
combined[["sporting", "loneliness"]] = gez[["WekelijkseSporters_6", "Eenzaam_28"]]


In [96]:
combined.head()

,gwb_code_8,regio,gm_naam,recs,gwb_code,ind_wbi,a_inw,a_man,a_vrouw,a_00_14,...,g_3km_sc,a_opp_ha,a_lan_ha,a_wat_ha,pst_mvp,pst_dekp,ste_mvs,ste_oad,sporting,loneliness
gwb_code_10,,,,,,,,,,,,,,,,,,,,,
NL00,0,Nederland,Nederland,Land,NL00,.,17282163,8581086,8701077,2739819,...,"10,9",4154338,3367109,787228,.,.,2,1994,NaN,NaN
GM0003,3,Appingedam,Appingedam,Gemeente,GM0003,.,11721,5720,6001,1718,...,"6,4",2458,2378,80,.,.,3,1067,36.3,49.4
WK000300,300,Wijk 00,Appingedam,Wijk,WK000300,1,11720,5720,6000,1715,...,"6,4",2458,2378,80,.,.,3,1067,36.3,49.4
BU00030000,30000,Appingedam-Centrum,Appingedam,Buurt,BU00030000,1,2370,1135,1230,230,...,"7,1",90,84,5,9901,1,3,1210,35.4,51.6
BU00030001,30001,Appingedam-West,Appingedam,Buurt,BU00030001,1,3035,1505,1530,480,...,"4,0",163,158,5,9903,6,4,903,41.5,41.5


In [97]:
# Drop the first row, as it contains data for the whole Netherlands.
combined = combined.drop("NL00", errors='ignore')

# List of columns in combined, that is not needed

columns_to_drop = ["gwb_code_8", "regio", "gm_naam", "recs", "gwb_code", "ind_wbi", "pst_mvp", "pst_dekp"]

combined = combined.drop(columns_to_drop, axis=1, errors='ignore')

# Missing Data


In [98]:
# Percent of missing data
print(round(combined.isna().sum().replace(0, np.nan).dropna() / len(combined) * 100, 3))

# Since we have data that are missing in the "predict" columns, we will drop those rows.
# Reason for having missing data there is most probably because of a missmatch in the gemeente codes from 2019 (kwb-dataset) and 2020 (gezondheid-dataset), and for the fact that some data was retracted for privacy reasons.
# This data was either way based on a small region, so it might even be good that it is removed.

combined = combined.dropna(subset=["sporting", "loneliness"])

a_inkont      100.000
g_ink_po      100.000
g_ink_pi      100.000
p_ink_li      100.000
p_ink_hi      100.000
p_n_act       100.000
p_hh_li       100.000
p_hh_hi       100.000
p_hh_lkk      100.000
p_hh_osm      100.000
g_wodief      100.000
g_vernoo      100.000
g_gewsek      100.000
g_afs_hp        3.939
g_afs_gs        3.939
g_afs_kv        3.939
ste_oad         0.012
sporting        5.762
loneliness      5.762
dtype: float64


In [99]:
# Get all the columns with the proportion of missing values
combined_missing_proportion = combined.isna().sum().replace(0, np.nan).dropna() / len(combined) * 100

# Drop the columns that have more than 50 percent missing values
columns_to_drop = combined_missing_proportion[combined_missing_proportion > 50].index
combined = combined.drop(columns=columns_to_drop)

# Percent of missing data
print(round(combined.isna().sum().replace(0, np.nan).dropna() / len(combined) * 100, 3))

g_afs_hp    0.211
g_afs_gs    0.211
g_afs_kv    0.211
dtype: float64


In [100]:
# After some further cleaning of the data we see that we now have around 0.211% missing values left to fix.
# We can see that where the data is missing, it is also missing in the two other columns.
# There is also a very small population, and since it represents less than 1% of the data, we will drop these rows as well. To not make the data we are using further cluttered with wrong data.
combined[["a_inw","g_afs_hp","g_afs_gs","g_afs_kv"]][combined["g_afs_hp"].isna()]

,a_inw,g_afs_hp,g_afs_gs,g_afs_kv
gwb_code_10,,,,
BU00140108,20,NaN,NaN,NaN
BU00140705,35,NaN,NaN,NaN
BU00807305,0,NaN,NaN,NaN
BU00981006,25,NaN,NaN,NaN
BU01140509,5,NaN,NaN,NaN
BU01770101,0,NaN,NaN,NaN
BU01800409,715,NaN,NaN,NaN
BU01932270,10,NaN,NaN,NaN
BU02690009,10,NaN,NaN,NaN


In [101]:
combined = combined.dropna(subset=["g_afs_hp","g_afs_gs","g_afs_kv"])
# No missing data left!
combined.isna().sum().sum()

0

In [102]:
combined.to_csv("pre-processed-data.csv")
